# import Libraries

In [62]:
import torch
from torch.utils.data import Dataset, DataLoader
from classes import SentimentDataset
import torch.optim as optim
load_path = 'sentiment_data_loader.pth' # Đảm bảo đường dẫn này chính xác
loaded_data = torch.load(load_path)
batch_size = loaded_data.get('batch_size', 32)
train_loader = loaded_data.get('train_loader')
test_loader = loaded_data.get('test_loader')
vocab = loaded_data.get('vocab')
vocab_size = loaded_data.get('vocab_size')

In [63]:
vocab_size

2307

# Build Model

In [64]:
# Phul luc B: model.py
import torch.nn as nn
import torchtext.vocab as tvocab
import numpy as np

# --- Helper function to load GloVe embeddings ---
def load_glove_embeddings(glove_path, vocab, embedding_dim):
    """
    Loads GloVe embeddings for words found in the vocabulary.

    Args:
        glove_path (str): Name of the GloVe vectors (e.g., 'glove.6B.100d').
                          Make sure you have downloaded these or torchtext can download them.
        vocab (dict): The vocabulary mapping words to indices.
        embedding_dim (int): The dimension of the GloVe embeddings.

    Returns:
        torch.Tensor: The embedding matrix.
    """
    print(f"Loading GloVe vectors: {glove_path}...")
    # Tải GloVe vectors sử dụng torchtext
    # Lần đầu chạy có thể mất thời gian để tải file GloVe
    try:
        glove = tvocab.GloVe(name=glove_path.split('.')[1], # e.g., '6B'
                             dim=embedding_dim,            # e.g., 100
                             cache='.vector_cache')        # Thư mục lưu cache
        print("GloVe vectors loaded successfully.")
    except Exception as e:
        print(f"Error loading GloVe vectors: {e}")
        print("Please ensure the GloVe files are available or can be downloaded.")
        print("You might need to install torchtext: pip install torchtext")
        # Hoặc tải thủ công từ: https://nlp.stanford.edu/projects/glove/
        # và giải nén vào thư mục .vector_cache
        raise e # Dừng chương trình nếu không tải được GloVe

    vocab_size = len(vocab)
    # Khởi tạo ma trận embedding với giá trị ngẫu nhiên nhỏ
    embeddings = np.random.uniform(-0.25, 0.25, (vocab_size, embedding_dim))
    embeddings[vocab['<PAD>']] = np.zeros(embedding_dim) # Vector 0 cho PAD

    # Điền vào ma trận embedding bằng vector GloVe nếu từ có trong GloVe
    loaded_count = 0
    for word, idx in vocab.items():
        if word in glove.stoi: # stoi: string-to-index mapping trong GloVe object

            embeddings[idx] = glove.vectors[glove.stoi[word]].numpy()
            loaded_count += 1
        # else: để lại giá trị khởi tạo ngẫu nhiên (hoặc có thể gán vector <UNK> nếu muốn)

    print(f"Loaded {loaded_count} vectors from GloVe out of {vocab_size} vocab size.")
    return torch.tensor(embeddings, dtype=torch.float)
# --------------------------------------------------
#pretrained_embeddings = load_glove_embeddings('glove.6B.100d',vocab,100)

In [65]:
class RNNModel(nn.Module):
    def __init__(self,vocab, vocab_size, embedding_dim, hidden_dim, output_dim,
                 pad_idx, pretrained=False, glove_path='glove.6B.100d'): # Thêm vocab và glove_path
        super().__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.vocab_size = vocab_size
        self.padding_idx = pad_idx # Lấy index của PAD token

        # --- Khởi tạo embedding layer ---
        if pretrained:
            print("Using pre-trained GloVe embeddings.")
            # Tải trọng số GloVe
            pretrained_embeddings = load_glove_embeddings(glove_path, vocab, embedding_dim)
            # Tạo lớp Embedding từ trọng số đã tải
            self.embedding = nn.Embedding.from_pretrained(
                pretrained_embeddings,
                freeze=False, # Cho phép fine-tuning embedding nếu muốn (False)
                padding_idx=self.padding_idx
            )
        else:
            print("Training embeddings from scratch.")
            # Khởi tạo embedding ngẫu nhiên
            self.embedding = nn.Embedding(
                num_embeddings=vocab_size,
                embedding_dim=embedding_dim,
                padding_idx=self.padding_idx
            )

        # --- Khởi tạo khối RNN layer ---
        # [Sinh viên bổ sung: dùng nn.RNN với batch_first=True]
        self.rnn = nn.RNN(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=1, # Giữ đơn giản với 1 lớp RNN
            batch_first=True, # Quan trọng: input/output có dạng (batch, seq, feature)
        )
        self.dropout = nn.Dropout(0.3)
        # --- Khởi tạo tầng Dense để dự đoán 3 nhãn ---
        # [Sinh viên bổ sung: dùng nn.Linear, nhận hidden state từ RNN]
        self.fc = nn.Linear(
            in_features=hidden_dim, # Input là hidden state cuối cùng của RNN
            out_features=output_dim # Output là số lớp cảm xúc (3)
        )
        # --- Hết phần bổ sung Dense ---

    def forward(self, text):
        # text shape: (batch_size, seq_len)

        # --- Chuyển text thành embedding ---
        # [Sinh viên bổ sung]
        # embedded shape: (batch_size, seq_len, embedding_dim)
        embedded = self.embedding(text)
        # --- Hết phần bổ sung embedding forward ---

        # --- Đưa qua khối RNN để lẩy hidden state cuối ---
        # [Sinh viên bổ sung]
        # output shape: (batch_size, seq_len, hidden_dim)
        # hidden shape: (num_layers, batch_size, hidden_dim) -> (1, batch_size, hidden_dim)
        rnn_output, hidden = self.rnn(embedded)
        #_, (hidden, _) = self.rnn(embedded)
        # Lấy hidden state cuối cùng của lớp RNN duy nhất
        # hidden.squeeze(0) loại bỏ chiều num_layers (vì = 1)
        # last_hidden shape: (batch_size, hidden_dim)
        #last_hidden = hidden.squeeze(0)
        last_hidden = hidden[-1] 
        # --- Hết phần bổ sung RNN forward ---

        # --- Đưa hidden state qua tầng Dense để dự đoán 3 nhãn ---
        # [Sinh viên bổ sung]
        # predictions shape: (batch_size, output_dim)
        last_hidden = self.dropout(last_hidden)  # <-- thêm dòng này

        predictions = self.fc(last_hidden)
        # --- Hết phần bổ sung Dense forward ---

        # [Sinh viên bổ sung: trả về kết quả dự đoán]
        return predictions


In [66]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, vocab, vocab_size, embedding_dim, hidden_dim, output_dim,
                 pad_idx, pretrained=False, glove_path='glove.6B.100d'): # Thêm vocab và glove_path
        super().__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.vocab_size = vocab_size
        self.padding_idx = pad_idx # Lấy index của PAD token

        # --- Khởi tạo embedding layer ---
        if pretrained:
            print("Using pre-trained GloVe embeddings.")
            # Tải trọng số GloVe
            pretrained_embeddings = load_glove_embeddings(glove_path, vocab, embedding_dim)
            # Tạo lớp Embedding từ trọng số đã tải
            self.embedding = nn.Embedding.from_pretrained(
                pretrained_embeddings,
                freeze=False, # Cho phép fine-tuning embedding nếu muốn (False)
                padding_idx=self.padding_idx
            )
        else:
            print("Training embeddings from scratch.")
            # Khởi tạo embedding ngẫu nhiên
            self.embedding = nn.Embedding(
                num_embeddings=vocab_size,
                embedding_dim=embedding_dim,
                padding_idx=self.padding_idx
            )

        # --- Khởi tạo khối LSTM layer ---
        # [Sinh viên bổ sung: dùng nn.LSTM với batch_first=True]
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=1, # Giữ đơn giản với 1 lớp LSTM
            batch_first=True, # Quan trọng: input/output có dạng (batch, seq, feature)
        )

        # --- Khởi tạo tầng Dense để dự đoán 3 nhãn ---
        self.fc = nn.Linear(
            in_features=hidden_dim, # Input là hidden state cuối cùng của LSTM
            out_features=output_dim # Output là số lớp cảm xúc (3)
        )

    def forward(self, text):
        # text shape: (batch_size, seq_len)

        # --- Chuyển text thành embedding ---
        embedded = self.embedding(text)
        # embedded shape: (batch_size, seq_len, embedding_dim)

        # --- Đưa qua khối LSTM để lấy hidden state cuối ---
        # output shape: (batch_size, seq_len, hidden_dim)
        # hidden shape: (num_layers, batch_size, hidden_dim) -> (1, batch_size, hidden_dim)
        lstm_output, (hidden, cell) = self.lstm(embedded)
        # hidden có shape (num_layers, batch_size, hidden_dim), ta chỉ cần hidden[-1]
        last_hidden = hidden[-1]  # LSTM output the last hidden state of the last layer

        # --- Đưa hidden state qua tầng Dense để dự đoán 3 nhãn ---
        predictions = self.fc(last_hidden)

        # Trả về kết quả dự đoán
        return predictions


# Train Model

In [67]:
# --- Configuration ---
embedding_dim = 100 # Kích thước vector embedding
hidden_dim = 128    # Kích thước lớp ẩn RNN
output_dim = 3
pad_idx = vocab["<PAD>"]

In [68]:
# # Giả sử vocab và vocab_size đã được import từ data.py
model_glove= RNNModel(vocab,vocab_size, embedding_dim, hidden_dim, output_dim,pad_idx, pretrained=False,glove_path='glove.6B.100d')
# # model_test_glove = RNNModel(vocab_size, embedding_dim_test, hidden_dim_test, output_dim_test, vocab, pretrained=True)
# # print(model_test_scratch)
# # print(model_test_glove)
# model_glove= LSTMModel(vocab,vocab_size, embedding_dim, hidden_dim, output_dim,pad_idx, pretrained=False,glove_path='glove.6B.100d')


Training embeddings from scratch.


In [69]:
len(train_loader)

322

In [70]:
import time
# --- Xác định thiết bị (CPU hoặc GPU nếu có) ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
learning_rate = 0.1 # Tốc độ học cho SGD
num_epochs = 100
# Khởi tạo mô hình
pad_idx = vocab['<PAD>']
# model = RNNModel(vocab_size, embedding_dim, hidden_dim, output_dim, pad_idx)
#model = model.to(device) # Chuyển model lên device
model = model_glove.to(device)
# --- Bước 3: Định nghĩa Loss và Optimizer ---
criterion = nn.CrossEntropyLoss() # Phù hợp cho bài toán phân loại đa lớp
optimizer = optim.SGD(model.parameters(), lr=learning_rate) # Sử dụng SGD theo yêu cầu

# --- Bước 4: Huấn luyện mô hình ---
print("\n--- Starting Training ---")
start_time = time.time()

for epoch in range(num_epochs):
    model.train() # Chuyển sang chế độ huấn luyện
    epoch_loss = 0
    epoch_correct = 0
    epoch_total = 0

    for batch_sequences, batch_labels in train_loader:
        # Chuyển dữ liệu batch lên device
        batch_sequences = batch_sequences.to(device)
        batch_labels = batch_labels.to(device)
        # 1. Xóa gradients cũ
        optimizer.zero_grad()

        # 2. Forward pass
        predictions = model(batch_sequences)


        # 3. Tính loss
        loss = criterion(predictions, batch_labels)
        

        # 4. Backward pass (tính gradient)
        loss.backward()

        # 5. Cập nhật trọng số
        optimizer.step()

        # Tính toán thống kê cho epoch
        epoch_loss += loss.item()
        preds_class = torch.argmax(predictions, dim=1)
        epoch_correct += (preds_class == batch_labels).sum().item()
        epoch_total += batch_labels.size(0)
    
    avg_epoch_loss = epoch_loss / len(train_loader)
    accuracy = epoch_correct / epoch_total
     # --- Validation sau mỗi epoch ---
    model.eval()  # Chuyển sang chế độ đánh giá (tắt dropout/batchnorm nếu có)
    val_loss = 0
    val_correct = 0
    val_total = 0

    with torch.no_grad():  # Tắt tính toán gradient để tiết kiệm bộ nhớ
        for val_sequences, val_labels in test_loader:
            val_sequences = val_sequences.to(device)
            val_labels = val_labels.to(device)

            val_predictions = model(val_sequences)
            loss = criterion(val_predictions, val_labels)

            val_loss += loss.item()
            preds_class = torch.argmax(val_predictions, dim=1)
            val_correct += (preds_class == val_labels).sum().item()
            val_total += val_labels.size(0)

    avg_val_loss = val_loss / len(test_loader)
    val_accuracy = val_correct / val_total
    # In kết quả sau mỗi vài epoch để theo dõi
    if (epoch + 1) % 10 == 0 or epoch == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {avg_epoch_loss:.4f}, Train Acc: {accuracy:.4f}, '
              f'Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.4f}')
end_time = time.time()
print(f"--- Training Finished ---")
print(f"Total Training Time: {end_time - start_time:.2f} seconds")

Using device: cpu

--- Starting Training ---
Epoch [1/100], Train Loss: 0.8429, Train Acc: 0.4984, Val Loss: 0.8312, Val Acc: 0.5101
Epoch [10/100], Train Loss: 0.7856, Train Acc: 0.6396, Val Loss: 0.9648, Val Acc: 0.5372
Epoch [20/100], Train Loss: 0.8291, Train Acc: 0.6371, Val Loss: 0.8717, Val Acc: 0.6737
Epoch [30/100], Train Loss: 0.8447, Train Acc: 0.6202, Val Loss: 0.8356, Val Acc: 0.6264
Epoch [40/100], Train Loss: 0.8370, Train Acc: 0.6708, Val Loss: 1.0908, Val Acc: 0.6474
Epoch [50/100], Train Loss: 2.3211, Train Acc: 0.5119, Val Loss: 1.0768, Val Acc: 0.5871
Epoch [60/100], Train Loss: 0.9665, Train Acc: 0.4857, Val Loss: 0.9130, Val Acc: 0.4803
Epoch [70/100], Train Loss: 0.9593, Train Acc: 0.4939, Val Loss: 0.9723, Val Acc: 0.4794
Epoch [80/100], Train Loss: 0.9715, Train Acc: 0.4866, Val Loss: 1.2365, Val Acc: 0.5066
Epoch [90/100], Train Loss: 0.9685, Train Acc: 0.4914, Val Loss: 1.0178, Val Acc: 0.4899
Epoch [100/100], Train Loss: 1.0337, Train Acc: 0.4883, Val Loss: 